In [13]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
Activities = [1,2,3,4,5,6,7,8,9,10,11]
import math



In [3]:
def getSamplesFromFiles(filesNames=["mHealth_ECGProcessed_subject3.log"],readingsPerSample=50):
    targets  = np.array([])
    feature_set= np.array([])
    for fileName in filesNames:
        df = pd.read_csv(fileName, header=None, delim_whitespace=True)
        for i in range(int(df.shape[0]/readingsPerSample)):
            if df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean() in Activities:
                feature_set=np.append(feature_set,[df[1][i*readingsPerSample:i*readingsPerSample+readingsPerSample]])
                targets=np.append(targets,df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean()-1)

    feature_set=feature_set.reshape(int(feature_set.shape[0]/readingsPerSample),readingsPerSample)
#     one_hot_labels=one_hot_labels.reshape(int(one_hot_labels.shape[0]/12),12)
    return feature_set, targets
def unison_shuffled_copies(a, b,seed=1):
    assert len(a) == len(b)
    np.random.seed(seed)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [47]:
feature_set,targets=getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log"],50)
feature_set,targets=unison_shuffled_copies(feature_set,targets)
X_train, X_test, y_train, y_test = train_test_split(feature_set, targets, test_size=0.33, random_state=10)

In [48]:
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train);

In [49]:
# Use the forest's predict method on the test data
y_pred = rf.predict_proba(X_test)
y_pred.argmax(1)
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred.argmax(1))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 33.23%


In [50]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [63]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 1000, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 25.7min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 34.9min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 44.3min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 51.8min finished
C:\Users\pcroot\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=1000, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [64]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 110,
 'bootstrap': False}

In [67]:
base_model = RandomForestClassifier()
base_model.set_params(**rf_random.best_params_)
base_model.fit(X_train, y_train)
# Use the forest's predict method on the test data
y_pred = base_model.predict_proba(X_test)
y_pred.argmax(1)
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred.argmax(1))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 35.05%


In [ ]:
""" best params, obtained in 1000 iterations with 3 fold cv
{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 110,
 'bootstrap': False}
"""